In [47]:
from langchain_community.document_loaders import TextLoader, WebBaseLoader, DirectoryLoader, PyPDFLoader
from dotenv import load_dotenv
load_dotenv()

True

## loading all files


In [ ]:
# # Load all text files from the directory
# loader = DirectoryLoader(
#     path="./data/text_data",
#     glob="**/*.txt",
#     loader_cls=TextLoader,
#     loader_kwargs={"encoding": "utf-8"},
#     show_progress=True

# )

# text_documents = loader.load()

# text_documents

In [ ]:
# print(f"Number of documents: {len(text_documents)}")

In [50]:
# # web loader
# web_loader = WebBaseLoader([
#     "https://en.wikipedia.org/wiki/Artificial_intelligence",
#     "https://en.wikipedia.org/wiki/Machine_learning",
#     "https://en.wikipedia.org/wiki/Deep_learning",
#     "https://en.wikipedia.org/wiki/Natural_language_processing",
#     "https://en.wikipedia.org/wiki/Computer_vision"
# ])
# web_docs = web_loader.load()

# web_docs

In [51]:
# len(web_docs)

In [ ]:
# from langchain_community.document_loaders import PyPDFDirectoryLoader

# pdf_loader = PyPDFDirectoryLoader(
#     path="./data/pdf_data",
#     glob="**/[!.]*.pdf",  # Default pattern
#     silent_errors=False,
#     recursive=False,
#     extract_images=True
# )

# pdf_documents = pdf_loader.load()
# pdf_documents

In [53]:
# len(pdf_documents)

## splitting all documents into chunks


In [54]:
# from langchain_classic.text_splitter import RecursiveCharacterTextSplitter

# # Combine all documents into one list
# all_documents = text_documents + pdf_documents + web_docs

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=200
# )


# all_chunks = text_splitter.split_documents(all_documents)

# print(f"Total chunks: {len(all_chunks)}")

In [55]:
# all_chunks[0]

## creating vector embeddings and storing


In [56]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

In [57]:
# embeddings = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-MiniLM-L12-v2",
#     model_kwargs={"device": "cpu"},

#     encode_kwargs={
#         "normalize_embeddings": True,  # ADD THIS - Important for similarity!
#         "batch_size": 32  # Optional but recommended
#     },

#     cache_folder="./models",
#     show_progress=True

# )

In [58]:
embeddings = OllamaEmbeddings(
    model="mxbai-embed-large:latest",
    base_url="http://localhost:11434",  # Ollama server URL
    num_thread=10  # Number of threads for processing
)

In [59]:
embeddings

OllamaEmbeddings(model='mxbai-embed-large:latest', validate_model_on_init=False, base_url='http://localhost:11434', client_kwargs={}, async_client_kwargs={}, sync_client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, keep_alive=None, num_thread=10, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

In [60]:
# vectorstore = Chroma.from_documents(
#     documents=all_chunks,
#     embedding=embeddings,
#     collection_name="rag_collection",
#     persist_directory="./chroma_db"  # Saves to disk
# )

In [61]:
# 2. Load existing vector store (NO re-embedding needed!)
vectorstore = Chroma(
    collection_name="rag_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_db"  # Points to existing folder
)

In [62]:
vectorstore

## creating retriver


In [63]:
# Create a retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# Use in your RAG pipeline
# retrieved_docs = retriever.invoke("What is deep learning?")
# len(retrieved_docs)

In [64]:
# retrieved_docs[0].page_content

## llm


In [65]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGroq(model="openai/gpt-oss-120b", temperature=0.2)

In [66]:
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before answering.

<context>
{context}
</context>

Question: {input}
""")

## building chain


In [67]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [68]:
document_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, document_chain)

In [69]:
result = rag_chain.invoke(
    input={"input": "What is data?"}
)
print(result["answer"])

**Step‑by‑step reasoning**

1. The excerpt talks about “our data” and then immediately moves to the most common structure used to *wrangle* data – the **data frame**.  
2. It describes a data frame as *tabular*, meaning it is organized in **rows and columns** just like a spreadsheet.  
3. An example is shown: a Titanic passenger data frame that contains fields such as **Name, PClass, Age, Sex, Survived, SexCode**. Each row represents one passenger (an observation) and each column represents a particular attribute (a variable).  
4. From this description we can infer that **data** is the collection of factual information (values) that can be stored, displayed, and processed in such a structured form.

**Answer**

Data is the collection of factual information—values or observations—about something. In practice it is often organized in a tabular structure called a *data frame*, where each **row** represents an individual record (e.g., a passenger) and each **column** represents a specific